<h1 style="font-size: 24px;">Visualizations for stop and search/buglary data in Barnet</h1>



<h1 style="font-size: 24px;">Libraries</h1>

In [1]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import sqlite3

In [2]:
stopandsearchdf = pd.read_parquet('stopandsearch.paraquet')
stopandsearchdf['Outcome'].unique()

array(['Nothing found - no further action',
       'Offender given drugs possession warning', 'Suspect arrested',
       'Local resolution', 'Offender given penalty notice',
       'Suspect summonsed to court', 'Offender cautioned',
       'Article found - Detailed outcome unavailable',
       'Khat or Cannabis warning', 'Arrest',
       'A no further action disposal', 'Penalty Notice for Disorder',
       'Community resolution', 'Summons / charged by post',
       'Caution (simple or conditional)'], dtype=object)

<span style="font-size: 16px;">Even by taking the top 15 LSOAS, for count burglaries/stop and search, we cannot notice a time trend in neither of them.</span


<h1 style="font-size: 24px;">Possible outcomes for stop and search</h1>


In [ ]:
# Assuming you have the barnet_lsoa_ss DataFrame

# Filter the DataFrame by a specific year
#season = int(input("Enter the season to filter by: "))


#Input outcome for stop and search 

#filtered_df = barnet_lsoa_ss[barnet_lsoa_ss['season'] == season]

# Create the bubble map using plotly
#fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude', color='count_burlg',
                       # size='count_burlg', hover_data=['LSOA code', 'count_burlg'],
                       # mapbox_style='open-street-map', zoom=10)

# Customize the appearance of the map
#fig.update_layout(title=f"Bubble Map of Burglaries in Barnet ({season})",
                 # mapbox=dict(center=dict(lat=filtered_df['Latitude'].mean(),
                                      #    lon=filtered_df['Longitude'].mean())))

# Show the plot
#fig.show()

<span style="font-size: 16px;">Preprocessing stop and search data </span>  
<span style="font-size: 16px;">If u want to study a specific outcome for the stop and search data, pick one from above , uncoment lines 10 and 11 in the code cell bellow , and restart and run all notebook.


In [4]:
stopandsearchdf['Outcome'].unique()
grouped = stopandsearchdf.groupby(stopandsearchdf['Date'].dt.to_period('M')).size()

plt.plot(grouped.index, grouped.values)
plt.xlabel('Time')
plt.ylabel('Number of Stop and Search Incidents')
plt.title('Stop and Search Incidents Over Time')
plt.show()

AttributeError: Can only use .dt accessor with datetimelike values

In [5]:
# Convert 'Date' column to datetime format
stopandsearchdf['Date'] = pd.to_datetime(stopandsearchdf['Date'])

# Extract year and month into new columns
stopandsearchdf['Year'] = stopandsearchdf['Date'].dt.year
stopandsearchdf['Month'] = stopandsearchdf['Date'].dt.month

# Remove the 'Date' column
stopandsearchdf = stopandsearchdf.drop('Date', axis=1)


# Get multiple values to filter by
outcomes = input("Enter the stop and search outcomes to filter by (separated by commas): ").split(',')

# Apply the filter
stopandsearchdf = stopandsearchdf[stopandsearchdf['Outcome'].isin(outcomes)].copy()


Enter the stop and search outcomes to filter by (separated by commas): Offender given drugs possession warning, Khat or Cannabis warning


In [6]:
stopandsearchdf['season'] =  pd.cut(stopandsearchdf['Month'], bins=[0, 4, 8, 12], labels=[1, 2, 3])
stopandsearchdf

,Type,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Year,Month,season
178940,Person and Vehicle search,0.0,NaN,51.435698,-0.004938,Male,18-24,Not Stated (NS),White,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2016,5,2
178951,Person search,0.0,NaN,51.467243,-0.114064,Male,25-34,White - Any other White ethnic background (W9),White,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2016,6,2
178954,Person search,0.0,NaN,51.466435,-0.216447,Female,18-24,White - White British (W1),White,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2016,6,2
178960,Person and Vehicle search,0.0,NaN,51.452640,-0.101295,Male,25-34,Asian or Asian British - Bangladeshi (A3),Asian,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2016,6,2
178963,Person and Vehicle search,0.0,NaN,51.598456,-0.076015,Male,18-24,White - Any other White ethnic background (W9),White,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2016,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364428,Person search,0.0,NaN,51.411094,-0.307928,Female,18-24,Mixed/Multiple ethnic groups - White and Asian,Asian,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2017,12,3
364437,Person search,0.0,NaN,51.543688,-0.271028,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Black,Articles for use in criminal damage,Offender given drugs possession warning,NaN,NaN,2017,12,3
364439,Person and Vehicle search,0.0,NaN,51.505479,-0.055135,Male,25-34,Asian/Asian British - Bangladeshi,Asian,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2017,12,3
364441,Person and Vehicle search,0.0,NaN,51.505479,-0.055135,Male,18-24,Asian/Asian British - Bangladeshi,Asian,Controlled drugs,Offender given drugs possession warning,NaN,NaN,2017,12,3


In [7]:
# Read the GeoJSON file
lsoa_data = gpd.read_file("london.geojson")

# Convert the coordinates in the stop and search data to Point objects
stopandsearchdf['geometry'] = [Point(xy) for xy in zip(stopandsearchdf['Longitude'], stopandsearchdf['Latitude'])]
stopandsearchdf = gpd.GeoDataFrame(stopandsearchdf, geometry='geometry')

# Perform the spatial join
stopandsearchdf_with_lsoa = gpd.sjoin(stopandsearchdf, lsoa_data, how='left', op='within')

# Extract the LSOA code from the joined data
stopandsearchdf_with_lsoa['LSOA_code'] = stopandsearchdf_with_lsoa['lsoa11cd']

# Save the result or use it for further analysis
#stopandsearchdf_with_lsoa.to_csv("stopandsearch_with_lsoa.csv")


/Users/drugarobert/opt/anaconda3/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/Users/drugarobert/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/1_/1c03qm8s7hg966bpwbd0nfs80000gn/T/ipykernel_3181/1954923633.py:9: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  stopandsearchdf_with_lsoa = gpd.sjoin(stopandsearchdf, lsoa_data, how='left', op='within')


Stop and Search Data with LSOA code

In [8]:
stopandsearchdf_with_lsoa

,Type,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Object of search,...,geometry,index_right,FID,lsoa11cd,lsoa11nm,IMDRank,IMDDecil,Shape__Area,Shape__Length,LSOA_code
178940,Person and Vehicle search,0.0,NaN,51.435698,-0.004938,Male,18-24,Not Stated (NS),White,Controlled drugs,...,POINT (-0.00494 51.43570),3940.0,3941.0,E01003219,Lewisham 025B,15835.0,5.0,5.391714e+05,5240.733170,E01003219
178951,Person search,0.0,NaN,51.467243,-0.114064,Male,25-34,White - Any other White ethnic background (W9),White,Controlled drugs,...,POINT (-0.11406 51.46724),3783.0,3784.0,E01003060,Lambeth 011E,5258.0,2.0,2.198071e+05,2820.539105,E01003060
178954,Person search,0.0,NaN,51.466435,-0.216447,Female,18-24,White - White British (W1),White,Controlled drugs,...,POINT (-0.21645 51.46643),5278.0,5279.0,E01004608,Wandsworth 006E,18238.0,6.0,2.475887e+05,2861.926098,E01004608
178960,Person and Vehicle search,0.0,NaN,51.452640,-0.101295,Male,25-34,Asian or Asian British - Bangladeshi (A3),Asian,Controlled drugs,...,POINT (-0.10129 51.45264),3797.0,3798.0,E01003074,Lambeth 018E,11024.0,4.0,1.826714e+06,6430.977711,E01003074
178963,Person and Vehicle search,0.0,NaN,51.598456,-0.076015,Male,18-24,White - Any other White ethnic background (W9),White,Controlled drugs,...,POINT (-0.07601 51.59846),1050.0,1051.0,E01002095,Haringey 006D,2671.0,1.0,7.397885e+05,5819.654652,E01002095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364428,Person search,0.0,NaN,51.411094,-0.307928,Female,18-24,Mixed/Multiple ethnic groups - White and Asian,Asian,Controlled drugs,...,POINT (-0.30793 51.41109),3692.0,3693.0,E01002968,Kingston upon Thames 009C,19427.0,6.0,1.430641e+06,8251.465871,E01002968
364437,Person search,0.0,NaN,51.543688,-0.271028,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Black,Articles for use in criminal damage,...,POINT (-0.27103 51.54369),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364439,Person and Vehicle search,0.0,NaN,51.505479,-0.055135,Male,25-34,Asian/Asian British - Bangladeshi,Asian,Controlled drugs,...,POINT (-0.05514 51.50548),4974.0,4975.0,E01004297,Tower Hamlets 027C,25393.0,8.0,3.039877e+05,3134.190665,E01004297
364441,Person and Vehicle search,0.0,NaN,51.505479,-0.055135,Male,18-24,Asian/Asian British - Bangladeshi,Asian,Controlled drugs,...,POINT (-0.05514 51.50548),4974.0,4975.0,E01004297,Tower Hamlets 027C,25393.0,8.0,3.039877e+05,3134.190665,E01004297


<span style="font-size: 20px;">Focus on Barnet

In [9]:
london_lsoa = gpd.read_file('London.geojson')

# Filter the LSOA data for Barnet
barnet_lsoa = london_lsoa[london_lsoa['lsoa11nm'].str.contains('Barnet')]
df_burlg = pd.read_csv('burg_count_by_season_lsoa.csv')


In [10]:
df_burlg_Barnet = df_burlg[df_burlg['LSOA code'].isin(barnet_lsoa['lsoa11cd'].values)]
df_burlg_Barnet
df_burlg_Barnet.to_csv('burg_count_by_season_lsoa_Barnet.csv')

In [11]:
stopandsearchdf_with_lsoa_barnet = stopandsearchdf_with_lsoa[stopandsearchdf_with_lsoa['lsoa11cd'].isin(barnet_lsoa['lsoa11cd'].values)]


In [12]:
count_s_s = stopandsearchdf_with_lsoa_barnet.groupby(by = ['lsoa11cd', 'season'] , as_index=False).count()
count_s_s = count_s_s[['lsoa11cd', 'season', 'Latitude']]

In [13]:
count_s_s.columns = ['LSOA code', 'season', 'count_stop_search']
count_s_s

,LSOA code,season,count_stop_search
0,E01000116,1,0
1,E01000116,2,0
2,E01000116,3,3
3,E01000118,1,0
4,E01000118,2,0
...,...,...,...
295,E01000321,2,0
296,E01000321,3,1
297,E01000322,1,0
298,E01000322,2,1


In [14]:
merged_df = count_s_s.merge(df_burlg_Barnet, on=['LSOA code', 'season'])
merged_df.columns = ['LSOA code', 'season', 'count_stop_search', 'count_burlg']

merged_df

,LSOA code,season,count_stop_search,count_burlg
0,E01000116,1,0,91
1,E01000116,2,0,61
2,E01000116,3,3,78
3,E01000118,1,0,66
4,E01000118,2,0,61
...,...,...,...,...
295,E01000321,2,0,78
296,E01000321,3,1,75
297,E01000322,1,0,86
298,E01000322,2,1,62


In [23]:
sum(merged_df[merged_df['season'] == 1]['count_stop_search'])

27

In [ ]:
df = merged_df
# Assuming you have a DataFrame called 'df' with columns: lsoa11cd, Year, count_stop_search, count_burlg
plt.figure(figsize=(10, 6))  # Set the figure size as desired

bar_width = 0.35  # Width of the bars
index = df['season']  # X-axis values

plt.bar(index, df['count_stop_search'], bar_width, label='Count Stop Search')
plt.bar(index + bar_width, df['count_burlg'], bar_width, label='Count Burlg')

plt.xlabel('Season')
plt.ylabel('Count')
plt.title('Count Stop Search vs Count Burlg over Seasons')
plt.legend()




plt.show()

In [ ]:
correlation = df['count_stop_search'].corr(df['count_burlg'])
print("Correlation between count_stop_search and count_burlg:", correlation)


Time trends for counts

In [ ]:
sorted_df_burlg_Barnet = merged_df.sort_values(by = 'count_burlg', ascending=False)

In [ ]:
top_lsoas_burlg = sorted_df_burlg_Barnet.head(15)

In [ ]:
top_lsoas_ss =  (merged_df.sort_values(by = 'count_burlg', ascending=False)).head(15)

In [ ]:


plt.figure(figsize=(10, 6))  

# Line plot for count_burlg
plt.plot(top_lsoas_burlg['season'].to_numpy(), top_lsoas_burlg['count_burlg'].to_numpy(), marker='o')

plt.xlabel('Season')
plt.ylabel('Count')
plt.title('Time Trend for Count of Burglaries')

plt.show()


In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size as desired

# Line plot for count_burlg
plt.plot(top_lsoas_ss['season'].to_numpy(), top_lsoas_ss['count_stop_search'].to_numpy(), marker='o')

plt.xlabel('Season')
plt.ylabel('Count')
plt.title('Time Trend for Count of Stop and Search')

plt.show()


<h1 style="font-size: 24px;">Conclusion Time Trends</h1>
<span style="font-size: 16px;">Even by taking the top 15 LSOAS, for count burglaries/stop and search, we cannot notice a time trend in neither of them.</span>


In [ ]:
# Calculate the lagged count_stop_search column
merged_df['lagged_count_stop_search'] = merged_df['count_stop_search'].shift()

# Calculate the time trend
merged_df['time_trend'] = merged_df['count_burlg'] - merged_df['lagged_count_stop_search']

# Plot the time trend
merged_df.plot(x='season', y='time_trend', kind='line')


In [ ]:
correlation = merged_df['lagged_count_stop_search'].corr(merged_df['count_burlg'])
print('Correlation:', correlation)

In [ ]:
stop_seach_burlg_Barnet = merged_df

In [ ]:
stop_seach_burlg_Barnet

In [ ]:
lsoa_coord = stopandsearchdf_with_lsoa[['LSOA_code', 'Latitude', 'Longitude' ]]
lsoa_coord.columns = ['LSOA code', 'Latitude', 'Longitude' ]
lsoa_coord.dropna()

In [ ]:
barnet_lsoa_ss = stop_seach_burlg_Barnet.merge(lsoa_coord, on='LSOA code')


In [ ]:
barnet_lsoa_ss

In [ ]:
stopandsearchdf['Outcome'].unique()

In [ ]:
# Assuming you have the barnet_lsoa_ss DataFrame

# Filter the DataFrame by a specific year
season = int(input("Enter the season to filter by: "))
#Input outcome for stop and search 

filtered_df = barnet_lsoa_ss[barnet_lsoa_ss['season'] == season]

# Create the bubble map using plotly
fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude', color='count_stop_search',
                        size='count_stop_search', hover_data=['LSOA code', 'count_stop_search'],
                        mapbox_style='open-street-map', zoom=10)

# Customize the appearance of the map
fig.update_layout(title=f"Bubble Map of Stop Searches in Barnet ({season})",
                  mapbox=dict(center=dict(lat=filtered_df['Latitude'].mean(),
                                          lon=filtered_df['Longitude'].mean())))

# Show the plot
fig.show()

In [ ]:

barnet_lsoa_ss[barnet_lsoa_ss['LSOA code'] == 'E01000242']

In [ ]:
# Assuming you have the barnet_lsoa_ss DataFrame

# Filter the DataFrame by a specific year
year = int(input("Enter the year to filter by: "))
#Input outcome for stop and search 

filtered_df = barnet_lsoa_ss[barnet_lsoa_ss['year'] == year]
filtered_df = filtered_df[filtered_df['LSOA code'] == 'E01000242']

# Create the bubble map using plotly
fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude', color='count_stop_search',
                        size='count_stop_search', hover_data=['LSOA code', 'count_stop_search'],
                        mapbox_style='open-street-map', zoom=10)

# Customize the appearance of the map
fig.update_layout(title=f"Bubble Map of Stop Searches in Barnet ({year})",
                  mapbox=dict(center=dict(lat=filtered_df['Latitude'].mean(),
                                          lon=filtered_df['Longitude'].mean())))

# Show the plot
fig.show()

In [ ]:
# Assuming you have the barnet_lsoa_ss DataFrame

# Filter the DataFrame by a specific year
season = int(input("Enter the season to filter by: "))


#Input outcome for stop and search 

filtered_df = barnet_lsoa_ss[barnet_lsoa_ss['season'] == season]

# Create the bubble map using plotly
fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude', color='count_burlg',
                        size='count_burlg', hover_data=['LSOA code', 'count_burlg'],
                        mapbox_style='open-street-map', zoom=10)

# Customize the appearance of the map
fig.update_layout(title=f"Bubble Map of Burglaries in Barnet ({season})",
                  mapbox=dict(center=dict(lat=filtered_df['Latitude'].mean(),
                                          lon=filtered_df['Longitude'].mean())))

# Show the plot
fig.show()